March Madness prediction: 2017,2018,2019

We tried several approches, but we observed that using only the Seed difference between the team as a single feature vector to predic the winning probability of a team gives better performance than using others features:( score difference,location, mean value ofthe features, etc)
    

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier # Import Random Forest Classifier\n",
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split # Import train_test_split function\n",
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation\n",
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [55]:
#import data prior to the current year data
# for 2017 and 2018 : need to change the data_dir
data_dir = 'C:\\Users\\Shibly\\Desktop\\Spring2020\\CS_895\\mens-machine-learning-competition-2019\\DataFiles\\'
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'NCAATourneyCompactResults.csv')

In [56]:
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [57]:
df_tour.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [58]:
def seed_to_int(seed):
    #Get just the digits from the seeding. Return as int
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label
df_seeds.head()

,Season,TeamID,seed_int
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [59]:
df_tour.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace=True, axis=1)
df_tour.head()

,Season,WTeamID,LTeamID
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325


In [60]:
#Add the seed difference 
df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
df_dummy = pd.merge(left=df_tour, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_dummy, right=df_lossseeds, on=['Season', 'LTeamID'])
df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
df_concat.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,SeedDiff
0,1985,1116,1234,9,8,1
1,1985,1120,1345,11,6,5
2,1985,1207,1250,1,16,-15
3,1985,1229,1425,9,8,1
4,1985,1242,1325,3,14,-11


In [61]:
# creating single vector features
df_wins = pd.DataFrame()
df_wins['SeedDiff'] = df_concat['SeedDiff']
df_wins['Result'] = 1

df_losses = pd.DataFrame()
df_losses['SeedDiff'] = -df_concat['SeedDiff']
df_losses['Result'] = 0

df_predictions = pd.concat((df_wins, df_losses))
df_predictions.head()

,SeedDiff,Result
0,1,1
1,5,1
2,-15,1
3,1,1
4,-11,1


In [62]:
#creating the dataset
X_train = df_predictions.SeedDiff.values.reshape(-1,1)
y_train = df_predictions.Result.values


In [63]:
# Split dataset into training set and test set\n",
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

In [64]:
print('Logistic Regression Classifier:')

Logistic Regression Classifier:


In [65]:
logreg = LogisticRegression()
params = {'C': np.logspace(start=-5, stop=3, num=9)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

Best log_loss: -0.5576, with best C: 1.0


C:\Users\Shibly\AppData\Local\conda\conda\envs\Keras2\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Shibly\AppData\Local\conda\conda\envs\Keras2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Shibly\AppData\Local\conda\conda\envs\Keras2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Shibly\AppData\Local\conda\conda\envs\Keras2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Shib

In [66]:
# Model Accuracy, how often is the classifier correct?\n",
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7162471395881007


In [67]:
#Preparing dataset for stage2 
data_dir2='C:\\Users\\Shibly\\Desktop\\Spring2020\\CS_895\\mens-machine-learning-competition-2019\\'
df_sample_sub = pd.read_csv(data_dir2 + 'SampleSubmissionStage2.csv')
n_test_games = len(df_sample_sub)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

In [68]:
# making dataset for stage2 prediciton
data_dir = 'C:\\Users\\Shibly\\Desktop\\Spring2020\\CS_895\\mens-machine-learning-competition-2019\\Stage2DataFiles\\'
df_seeds2 = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
df_seeds2['seed_int'] = df_seeds2.Seed.apply(seed_to_int)
df_seeds2.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label
df_seeds2.head()

,Season,TeamID,seed_int
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [79]:
# predicitng the winning probability using logistic regression
X_test2 = np.zeros(shape=(n_test_games, 1))
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_seed = df_seeds2[(df_seeds2.TeamID == t1) & (df_seeds2.Season == year)].seed_int.values[0]
    t2_seed = df_seeds2[(df_seeds2.TeamID == t2) & (df_seeds2.Season == year)].seed_int.values[0]
    diff_seed2 = t1_seed - t2_seed
    X_test2[ii, 0] = diff_seed2
preds = clf.predict_proba(X_test2)[:,1]

clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = clipped_preds
df_sample_sub.head()


,ID,Pred
0,2019_1101_1113,0.334362
1,2019_1101_1120,0.158546
2,2019_1101_1124,0.265929
3,2019_1101_1125,0.334362
4,2019_1101_1133,0.491297


In [80]:
#creating the submission file

df_sample_sub.to_csv('2019_logreg.csv', index=False)

#creating the bracket usong bracketer 

from bracketeer import build_bracket 

m = build_bracket(outputPath = 'bracket_lg_19.png',
	teamsPath = r'C:\Users\Shibly\Desktop\Spring2020\CS_895\mens-machine-learning-competition-2019\Stage2DataFiles\Teams.csv',
	seedsPath = r'C:\Users\Shibly\Desktop\Spring2020\CS_895\mens-machine-learning-competition-2019\Stage2DataFiles\NCAATourneySeeds.csv',
	submissionPath = r'C:\Users\Shibly\logreg_seed_starter_2019_logreg.csv',
	slotsPath = r'C:\Users\Shibly\Desktop\Spring2020\CS_895\mens-machine-learning-competition-2019\Stage2DataFiles\NCAATourneySlots.csv',
	year = 2019)

# output feedback
print()
print()
print("2019 Bracket creation using lg complete.")
print()
print()



2019 Bracket creation using lg complete.




In [81]:
print('Random Forest Classifier:')
# Split dataset into training set and test set\n",
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=1) # 70% training and 30% test"

Random Forest Classifier:


In [82]:
# Create Random Forest classifer object\n",
Rf = RandomForestClassifier(n_estimators=2000, random_state=0)
# Train Random Forest Classifer\n",
Rf = Rf.fit(X_train,y_train)
#Predict the response for test dataset\n",
y_pred = Rf.predict(X_test)

In [83]:
# Model Accuracy, how often is the classifier correct?\n",
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6962616822429907


In [84]:
#Predicing probability of winning using Random forest Classifier
preds = Rf.predict_proba(X_test2)[:,1]

clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = clipped_preds
df_sample_sub.head()

,ID,Pred
0,2019_1101_1113,0.293028
1,2019_1101_1120,0.491712
2,2019_1101_1124,0.253947
3,2019_1101_1125,0.293028
4,2019_1101_1133,0.510272


In [165]:
df_sample_sub.to_csv('2019_RF.csv', index=False)

In [85]:
#creating submission file 
df_sample_sub.to_csv('2019_RF.csv', index=False)

# creating brackets
from bracketeer import build_bracket 

m = build_bracket(outputPath = 'bracket_RF_19.png',
	teamsPath = r'C:\Users\Shibly\Desktop\Spring2020\CS_895\mens-machine-learning-competition-2019\Stage2DataFiles\Teams.csv',
	seedsPath = r'C:\Users\Shibly\Desktop\Spring2020\CS_895\mens-machine-learning-competition-2019\Stage2DataFiles\NCAATourneySeeds.csv',
	submissionPath = r'C:\Users\Shibly\2019_RF.csv',
	slotsPath = r'C:\Users\Shibly\Desktop\Spring2020\CS_895\mens-machine-learning-competition-2019\Stage2DataFiles\NCAATourneySlots.csv',
	year = 2019)

# output feedback
print()
print()
print("2019 Bracket creation using RF complete.")
print()
print()



2019 Bracket creation using RF complete.




In [88]:
##svm
print('Support vector machine Classifier')
# Split dataset into training set and test set\n",
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=6) # 70% training and 30% test"

Support vector machine Classifier


In [89]:
 # Create SVM\n",
svc = LinearSVC()
# Train SVM\n",
svc = svc.fit(X_train,y_train)
#Predict the response for test dataset\n",
y_pred = svc.predict(X_test)
# Model Accuracy, how often is the classifier correct?\n",
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7206349206349206


C:\Users\Shibly\AppData\Local\conda\conda\envs\Keras2\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [90]:
preds = svc._predict_proba_lr(X_test2)[:,1]

clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = clipped_preds
df_sample_sub.head()

,ID,Pred
0,2019_1101_1113,0.433775
1,2019_1101_1120,0.343422
2,2019_1101_1124,0.402834
3,2019_1101_1125,0.433775
4,2019_1101_1133,0.496986


In [ ]:
# creating submission file 
df_sample_sub.to_csv('2019_SVM.csv', index=False)

#creating bracket

from bracketeer import build_bracket 

m = build_bracket(outputPath = 'bracket_svm_19.png',
	teamsPath = r'C:\Users\Shibly\Desktop\Spring2020\CS_895\mens-machine-learning-competition-2019\Stage2DataFiles\Teams.csv',
	seedsPath = r'C:\Users\Shibly\Desktop\Spring2020\CS_895\mens-machine-learning-competition-2019\Stage2DataFiles\NCAATourneySeeds.csv',
	submissionPath = r'C:\Users\Shibly\2019_SVM.csv',
	slotsPath = r'C:\Users\Shibly\Desktop\Spring2020\CS_895\mens-machine-learning-competition-2019\Stage2DataFiles\NCAATourneySlots.csv',
	year = 2019)

# output feedback
print()
print()
print("2019 Bracket creation using SVM complete.")
print()
print()